# Big Data Modeling and Management Assigment - Homework 1

## 🍺 The Beer project  🍺 

As it was shown in classes, graph databases are a natural way of navegating distinct types of data. For this first project we will be taking a graph database to analyse beer and breweries!   

_For reference the dataset used for this project has been extracted from [kaggle](https://www.kaggle.com/ehallmar/beers-breweries-and-beer-reviews), released by Evan Hallmark. Even though the author does not present metada on the origin of the data it is probably a collection of open data from places like [beeradvocate](https://www.beeradvocate.com/)_.

### Problem description

Explore the database via python neo4j connector and/or the graphical tool in the NEO4J webpage. Answer the questions. Submit the results by following the instructions


### Questions

1. How many different countries exist in the database?
1. Most reviews:  
    1. Which `Beer` has the most reviews?  
    1. Which `Brewery` has the most reviews for its beers? [Hint: 5-node path]
    1. Which `Country` has the most reviews for its beers? [Hint: 5-node path]
1. Find the user/users that have the most shared reviews (reviews of the same beers) with the user CTJman?
1. Which Portuguese brewery has the most beers?
1. From those beers (the ones returned from the previous question), which has the most reviews?
1. On average how many different beer styles does each brewery produce?
1. Which brewery produces the strongest beers according to ABV? [Hint: database has NaN values]
1. If I typically enjoy a beer due to its aroma and appearance, which beer style should I try? (Justify your answer well!) [Hint: database has NaN values]
1. Using Graph Algorithms answer **one** of the following questions: [Hint: make sure to clear the graph before using it again]
    1. Which two countries are most similiar when it comes to their **top 10** most produced Beer styles?
    2. Which beer is the most influential when considering beers are conected by users who review them? [Please use limit of 1000 on beer-review-user path]]
    3. Users are connected together by their reviews to beers, taking into consideration the "overall" score they review as a weight, how many communities are formed from these relationships? How many users has the biggest community? [Please use limit of 1000 on beer-review-user path]]
1. Using Graph Algorithms answer **one** of the following questions:
    1. Which beer has the most similar reviews as the beer `Super Bock Stout`? [Hint:inspect two subsets: with and without the beer in question]
    2. Which user is the most influential when it comes to reviews made?
1. If you had to pick 3 beers to recommend using only this database, which would you pick and why? (Justify your answer well!) [Hint: database has NaN values]


Questions 8 to 10 are somewhat open, which means we'll also be evaluating the reasoning behind your answer. So there aren't necessarily bad results there are only wrong criteria, explanations or execution. 
 
### Groups  

Groups should have 4 to 5 people.
You should register your group on **moodle**.

### Submission      

The code used to produce the results and respective explations should be uploaded to moodle. They should have a clear reference to the group, either on the file name or on the document itself. Preferably one Jupyter notebook per group.

Delivery date: Until the **midnight of March 13**

### Evaluation   

This will be 20% of the final grade.   
Each solution will be evaluated on 2 components: correctness of results and simplicity of the solution.  
All code will go through plagiarism automated checks. Groups with the same code will undergo investigation.

## Loading the Database

#### Be sure that you **don't have** the neo4j docker container from the classes running (you can Stop it in the desktop app or with the command "`docker stop Neo4JLab`")


The default container does not have any data whatsoever, we will have to load a database into our docker image:
- Download and unzip the `Neo4JHWData` file provided in Moodle.
- Copy the path of the `Neo4JHWData` folder of the unziped file, e.g. `C:/Users/nunoa/Desktop/Aulas/Big Data Management and Modelling/2023/HW1/Neo4JHWData/data`.
- Download and unzip the `Neo4JPlugins` file provided in Moodle.
- Copy the path of the `Neo4JPlugins` folder of the unziped file, e.g. `C:/Users/nunoa/Desktop/Aulas/Big Data Management and Modelling/2023/Neo4Jplugins`.
- Change the code bellow accordingly. As you might have noticed, you do not have a user called `nunoa`, please use the appropriate path that you got from the previous step. Be sure that you have a neo4j docker container running: \

`docker run --name Neo4JHW -p 7474:7474 -p 7687:7687 -d -v "C:/Users/nunoa/Desktop/Aulas/Big Data Management and Modelling/2023/Neo4Jplugins":/plugins -v "C:/Users/nunoa/Desktop/Aulas/Big Data Management and Modelling/2023/HW1/Neo4JHWData/data":/data --env NEO4J_AUTH=neo4j/test --env NEO4J_dbms_connector_https_advertised__address="localhost:7473" --env NEO4J_dbms_connector_http_advertised__address="localhost:7474" --env NEO4J_dbms_connector_bolt_advertised__address="localhost:7687" --env NEO4J_dbms_security_procedures_unrestricted=gds.* --env NEO4J_dbms_security_procedures_allowlist=gds.* --env NEO4J_dbms_memory_heap_max__size="6g" neo4j:4.4.5`

- Since Neo4j is trying to recognize a new database folder, this might take a bit (let's say 3 minutes), so don't worry.

In [2]:
from neo4j import GraphDatabase
from pprint import pprint

In [3]:
NEO4J_URI="neo4j://localhost:7687"
NEO4J_USERNAME="neo4j"
NEO4J_PASSWORD="test"

In [4]:
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD), )

### All The Functions you'll need to run queries in Neo4J

In [5]:
def execute_read(driver, query):    
    with driver.session(database="neo4j") as session:
        result = session.execute_read(lambda tx, query: list(tx.run(query)), query)
    return result

## 1. Understanding the Database

### 1.1. First Look at the Nodes and Relationship Types

In [6]:
query = """
        call db.labels();
    """

result = execute_read(driver, query)

pprint(result)

[<Record label='COUNTRIES'>,
 <Record label='CITIES'>,
 <Record label='BREWERIES'>,
 <Record label='BEERS'>,
 <Record label='REVIEWS'>,
 <Record label='STYLE'>,
 <Record label='USER'>]


In [7]:
query = """
        CALL db.relationshipTypes();
    """

result = execute_read(driver, query)

pprint(result)

[<Record relationshipType='REVIEWED'>,
 <Record relationshipType='BREWED'>,
 <Record relationshipType='IN'>,
 <Record relationshipType='HAS_STYLE'>,
 <Record relationshipType='POSTED'>]


### 1.2. Checking the Relation between the Nodes and Relationship Types

In [8]:
# In the next few lines of code we are going to try to understand
# which relationship types are possibly connected to each node. This
# will give us a better sense of the data and how the database was
# built. Not only that, but we are also going to understand in which direction
# is the relationship type pointing.

In [9]:
# Checking if there is any relationship type pointing to the node "COUNTRIES".
# The relationship type "IN" points to the node "COUNTRIES".

query = """
        MATCH (c:COUNTRIES)<-[d]-()
        RETURN DISTINCT type(d)
    """

result = execute_read(driver, query)

pprint(result)

[<Record type(d)='IN'>]


In [10]:
# Checking if there is any relationship type pointing towards the opposite direction of the node "COUNTRIES".
# No relationship type points in the opposite direction of the node "COUNTRIES".

query = """
        MATCH (c:COUNTRIES)-[d]->()
        RETURN DISTINCT type(d)
    """

result = execute_read(driver, query)

pprint(result)

[]


In [11]:
# Checking if there is any relationship type pointing to the node "CITIES".
# The relationship type "IN" points to the node "CITIES".

query = """
        MATCH (c:CITIES)<-[d]-()
        RETURN DISTINCT type(d)
    """

result = execute_read(driver, query)

pprint(result)

[<Record type(d)='IN'>]


In [12]:
# Checking if there is any relationship type pointing towards the opposite direction of the node "CITIES".
# The relationship type "IN" points to the opposite direction of the node "COUNTRIES".

query = """
        MATCH (c:CITIES)-[d]->()
        RETURN DISTINCT type(d)
    """

result = execute_read(driver, query)

pprint(result)

[<Record type(d)='IN'>]


In [13]:
# Checking if there is any relationship type pointing to the node "BREWERIES".
# No relationship type points to the node "BREWERIES".

query = """
        MATCH (b:BREWERIES)<-[d]-()
        RETURN DISTINCT type(d)
    """

result = execute_read(driver, query)

pprint(result)

[]


In [14]:
# Checking if there is any relationship type pointing towards the opposite direction of the node "BREWERIES".
# The relationship type "IN" points to the opposite direction of the node "BREWERIES".

query = """
        MATCH (b:BREWERIES)-[d]->()
        RETURN DISTINCT type(d)
    """

result = execute_read(driver, query)

pprint(result)

[<Record type(d)='IN'>, <Record type(d)='BREWED'>]


In [15]:
# Checking if there is any relationship type pointing to the node "BEERS".
# The relationship type "BREWED" points to the node "BEERS".

query = """
        MATCH (b:BEERS)<-[d]-()
        RETURN DISTINCT type(d)
    """

result = execute_read(driver, query)

pprint(result)

[<Record type(d)='BREWED'>]


In [16]:
# Checking if there is any relationship type pointing towards the opposite direction of the node "BEERS".
# The relationship type "HAS_STYLE" and "REVIEWED" points to the opposite direction of the node "BEERS".

query = """
        MATCH (b:BEERS)-[d]->()
        RETURN DISTINCT type(d)
    """

result = execute_read(driver, query)

pprint(result)

[<Record type(d)='HAS_STYLE'>, <Record type(d)='REVIEWED'>]


In [17]:
# Checking if there is any relationship type pointing to the node "REVIEWS".
# The relationship type "REVIEWED" points to the node "REVIEWS".

query = """
        MATCH (r:REVIEWS)<-[d]-()
        RETURN DISTINCT type(d)
    """

result = execute_read(driver, query)

pprint(result)

[<Record type(d)='REVIEWED'>]


In [18]:
# Checking if there is any relationship type pointing towards the opposite direction of the node "REVIEWS".
# The relationship type "POSTED" points to the opposite direction of the node "REVIEWS".

query = """
        MATCH (r:REVIEWS)-[d]->()
        RETURN DISTINCT type(d)
    """

result = execute_read(driver, query)

pprint(result)

[<Record type(d)='POSTED'>]


In [19]:
# Checking if there is any relationship type pointing to the node "STYLE".
# The relationship type "HAS_STYLE" points to the node "STYLE".

query = """
        MATCH (s:STYLE)<-[d]-()
        RETURN DISTINCT type(d)
    """

result = execute_read(driver, query)

pprint(result)

[<Record type(d)='HAS_STYLE'>]


In [20]:
# Checking if there is any relationship type pointing towards the opposite direction of the node "STYLE".
# No relationship type points in the opposite direction of the node "STYLE".

query = """
        MATCH (s:STYLE)-[d]->()
        RETURN DISTINCT type(d)
    """

result = execute_read(driver, query)

pprint(result)

[]


In [21]:
# Checking if there is any relationship type pointing to the node "USER".
# The relationship type "POSTED" points to the node "USER".

query = """
        MATCH (u:USER)<-[d]-()
        RETURN DISTINCT type(d)
    """

result = execute_read(driver, query)

pprint(result)

[<Record type(d)='POSTED'>]


In [22]:
# Checking if there is any relationship type pointing towards the opposite direction of the node "USER".
# No relationship type points in the opposite direction of the node "USER".

query = """
        MATCH (u:USER)-[d]->()
        RETURN DISTINCT type(d)
    """

result = execute_read(driver, query)

pprint(result)

[]


In [23]:
# From our initial analysis we were able to understand all the relations
# between each node and their respective relationship types, with them being
# presented in the image below.

<img src="Organogram.png" style="width:60%; height:100%;">

### 1.3. Checking the Different Properties of Each Node

In [24]:
# The node "COUNTRIES" will have "name" as its properties.

query = """
        MATCH (c:COUNTRIES)
        RETURN c
        LIMIT 1
    """

result = execute_read(driver, query)

pprint(result)

[<Record c=<Node element_id='0' labels=frozenset({'COUNTRIES'}) properties={'name': 'BE'}>>]


In [25]:
# The node "CITIES" will have "name" as its properties.

query = """
        MATCH (c:CITIES)
        RETURN c
        LIMIT 1
    """

result = execute_read(driver, query)

pprint(result)

[<Record c=<Node element_id='200' labels=frozenset({'CITIES'}) properties={'name': 'Erpe-Mere'}>>]


In [26]:
# The node "BREWERIES" will have "types", "notes", "name", "id", and "state" as its properties.

query = """
        MATCH (b:BREWERIES)
        RETURN b
        LIMIT 1
    """

result = execute_read(driver, query)

pprint(result)

[<Record b=<Node element_id='11865' labels=frozenset({'BREWERIES'}) properties={'types': 'Brewery', 'notes': 'No notes at this time.', 'name': 'Brouwerij Danny', 'id': '19730', 'state': 'nan'}>>]


In [27]:
# The node "BREWERIES" will have "notes", "abv", "name", "retired","state",
# "id", "availability", "brewery_id" as its properties.

query = """
        MATCH (b:BEERS)
        RETURN b
        LIMIT 1
    """

result = execute_read(driver, query)

pprint(result)

[<Record b=<Node element_id='62212' labels=frozenset({'BEERS'}) properties={'notes': 'No notes at this time.', 'abv': '7.3', 'name': 'Olde Cogitator', 'retired': 'f', 'state': 'CA', 'id': '202522', 'availability': ' Rotating', 'brewery_id': '2199'}>>]


In [28]:
# The node "STYLE" will have "name" as its properties.

query = """
        MATCH (s:STYLE)
        RETURN s
        LIMIT 1
    """

result = execute_read(driver, query)

pprint(result)

[<Record s=<Node element_id='9494213' labels=frozenset({'STYLE'}) properties={'name': 'English Oatmeal Stout'}>>]


In [83]:
# The node "REVIEWS" will have "date", "score", "taste", "feel", "overall",
# "beer_id", "text", "id", "smell", "look" as its properties.

query = """
        MATCH (r:REVIEWS)
        RETURN r
        LIMIT 1
    """

result = execute_read(driver, query)

pprint(result)

[<Record r=<Node element_id='421085' labels=frozenset({'REVIEWS'}) properties={'date': '2017-03-17', 'score': '4.03', 'taste': '4.0', 'feel': '4.25', 'overall': '4.0', 'beer_id': '271781', 'text': "\xa0\xa0 750 ml bottle, 2016 vintage, bottle #304 of 360. Served in a DFH snifter, the beer pours dark brown/black with about an inch tan head that stuck around a little while. There's also a good amount of lacing. I like the aroma, the brew smells like dark fruit (raisin, plum), chocolate, roasted malt, and a bit of licorice. For the most part I think the taste is just like the aroma, but with some coffee flavor noticeable too. Mouthfeel/body is medium/full, it's creamy and coating with a moderate amount of carbonation. I think it's a good overall brew, worth picking up if you want a nice straight forward imperial stout. Here's hoping Four String brings this back in the future! $9.99 a bottle. \xa0", 'id': '0', 'smell': '4.0', 'look': '4.0'}>>]


In [30]:
# The node "USER" will have "name" as its properties.

query = """
        MATCH (u:USER)
        RETURN u
        LIMIT 1
    """

result = execute_read(driver, query)

pprint(result)

[<Record u=<Node element_id='9494326' labels=frozenset({'USER'}) properties={'name': 'bluejacket74'}>>]


<img src="Properties.png" style="width:40%; height:100%;">

# Submission

GROUP NUMBER:

**22**

GROUP MEMBERS:

|STUDENT NUMBER|STUDENT NAME|
|---|---|
|20210581|Miguel Ramos|
|20221022|Eduardo Palma|
|20220607|José Matos|

#### 1. How many different countries exist in the database?

In [6]:
# Selecting the node "COUNTRIES" and counting all the distinct values
# associated with the property "name".

query = """
        MATCH (c:COUNTRIES)
        RETURN COUNT(DISTINCT c.name) as num_countries
    """

result = execute_read(driver, query)
num_countries = result[0]["num_countries"]

# We print the final answer.

print(f"There are {num_countries} different countries on this Graph Database.")

There are 200 different countries on this Graph Database.


#### 2. Most reviews:  
    A) Which `Beer` has the most reviews?  
    B) Which `Brewery` has the most reviews for its beers?
    C) Which `Country` has the most reviews for its beers? 

In [7]:
# Selecting both the nodes "BEERS" and "REVIEWS", including their relationship
# type "REVIEWED". Then, we are grouping by the name of the beer, and counting
# the number of reviews for each one of the grouped by beers.

query = """
        MATCH (b:BEERS)-[r1:REVIEWED]->(r2:REVIEWS)
        RETURN b.name as name_beer, COUNT(r2) as number_reviews
        ORDER BY number_reviews DESC
        LIMIT 1
    """

result = execute_read(driver, query)
name_beer = result[0]["name_beer"]
number_reviews = result[0]["number_reviews"]

# We print the final answer.

print(f"The beer that was reviewed the most was {name_beer}, with a total of {number_reviews} reviews.")

The beer that was reviewed the most was IPA, with a total of 31387 reviews.


In [39]:
# Selecting the three needed nodes, being them "BREWERIES", "BEERS", and "REVIEWS",
# and also selecting the relationship types that connect them. Then, we are grouping
# by the name of the brewery, and counting the total number of reviews the beers of each
# one of the breweries had.

query = """
        MATCH (b1:BREWERIES)-[b2:BREWED]->(b3:BEERS)-[r1:REVIEWED]->(r2:REVIEWS)
        RETURN b1.name as brewery_name, COUNT(r2) as number_reviews
        ORDER BY number_reviews DESC
        LIMIT 1
    """

result = execute_read(driver, query)
brewery_name = result[0]["brewery_name"]
number_reviews = result[0]["number_reviews"]

# We print the final answer.

print(f"The brewery that has the most beers reviewed is {brewery_name}, with a total of {number_reviews} reviews.")

The brewery that has the most beers reviewed is Sierra Nevada Brewing Co., with a total of 175161 reviews.


In [40]:
# To get a relation between the node "COUNTRIES" and "REVIEWS", we have to select
# these nodes and also all the nodes between them, these are "CITIES", "BREWERIES",
# "BEERS", and "REVIEWS". Not only that but we also have to select all the relationship
# types that allow to create connections between all these nodes. Then, we are grouping
# by the name of the country, and counting the number os reviews from all the beers from
# all the breweries of that specific country.

query = """
        MATCH (c1:COUNTRIES)<-[:IN]-(c2:CITIES)<-[:IN]-(b1:BREWERIES)-[:BREWED]->(b2:BEERS)-[:REVIEWED]->(r:REVIEWS)
        RETURN c1.name as country_name, COUNT(r) as number_reviews
        ORDER BY number_reviews DESC
        LIMIT 1
    """

result = execute_read(driver, query)
country_name = result[0]["country_name"]
number_reviews = result[0]["number_reviews"]

# We print the final answer.

print(f"The country that has the most beers reviewed is {country_name}, with a total of {number_reviews} reviews.")

The country that has the most beers reviewed is US, with a total of 7675804 reviews.


#### 3. Find the user/users that have the most shared reviews (reviews of the same beers) with the user CTJman?

In [41]:
# Initially finding out how many reviews have the user CTJman done. For that
# we have to select both the "BEERS" and "USER" nodes, this time already selecting
# the property name: "CTJman" as it is from him that we want to draw the information.
# We also select the relationship type "REVIEWS" that connects both nodes. We return
# the number of beers that were reviewed by counting the number of "BEERS" nodes.

query1 = """
        MATCH (b:BEERS)-[]->(r1:REVIEWS)-[]->(u1:USER{name:"CTJman"})
        RETURN COUNT(b) as number_beers1
        ORDER BY number_beers1 DESC
    """

result1 = execute_read(driver, query1)
number_beers1=result1[0]["number_beers1"]

# Then, we have to find the user that has most co-occurences related to beer reviews, that
# is, the user with the most shared reviews with CTJman. We can do that by selecting the same nodes
# we refered before but now we have to give two sides to it, one for the user "CTJman", and the other
# side for the user that we are looking for. We return a group by, by the name of the user
# which is ordered by the total number of shared reviews.


query2 = """
        MATCH (u2:USER)<-[]-(r2:REVIEWS)<-[]-(b:BEERS)-[]->(r1:REVIEWS)-[]->(u1:USER{name:"CTJman"})
        RETURN u2.name as name, COUNT(b) as number_beers2
        ORDER BY number_beers2 DESC
        LIMIT 1
    """

result2 = execute_read(driver, query2)
name=result2[0]["name"]
number_beers2=result2[0]["number_beers2"]

# We print the final answer.

print(f"Out of all the {number_beers1} beers that the user CTJman reviewed, {number_beers2} of them were shared reviews with the user {name}.")

Out of all the 2155 beers that the user CTJman reviewed, 1428 of them were shared reviews with the user acurtis.


#### 4. Which Portuguese brewery has the most beers?


In [42]:
# We will have to select the nodes that go all the way from the node "COUNTRIES" to the node "BEERS",
# and not only that but we will only want the sequences that start with the property name "PT" in the node
# "COUNTRIES", as we only want the Portuguese Breweries. We also have to select all the relationship types
# that connect the mentioned nodes. We return a group by, by the brewery name, and the respective number
# of beers of each of the breweries.

query = """
        MATCH (c1:COUNTRIES{name:"PT"})<-[]-(:CITIES)<-[]-(b1:BREWERIES)-[]->(b2:BEERS)
        RETURN b1.name as brewery_name, COUNT(b2) as number_beers
        ORDER BY number_beers DESC
        LIMIT 1
    """

result = execute_read(driver, query)
brewery_name=result[0]["brewery_name"]
number_beers=result[0]["number_beers"]

print(f"The Portuguese brewery that has the most beers is {brewery_name}, counting with {number_beers} beers.")

The Portuguese brewery that has the most beers is Dois Corvos Cervejeira, counting with 40 beers.


#### 5. From those beers (the ones produced in the brewery from the previous question), which has the most reviews?


In [43]:
# Selecting the "BREWERIES" nodes where the property name has the value of "Dois Corvos Cervejeira",
# then making the connection with the node "BEERS" and the node "REVIEWS", using the relationship
# types between them. Then, we group by each beer name, which is the property name of the node "BEERS" and
# we return the number of reviews for each beer name by counting the number of "REVIEWS" node (for each beer
# name).

query = """
        MATCH (b1:BREWERIES{name:"Dois Corvos Cervejeira"})-[]->(b2:BEERS)-[r1:REVIEWED]->(r2:REVIEWS)
        RETURN b2.name as beer_name, COUNT(r2) as number_reviews
        ORDER BY number_reviews DESC
        LIMIT 1
    """

result = execute_read(driver, query)
beer_name=result[0]["beer_name"]
number_reviews=result[0]["number_reviews"]

print(f"From the brewery Dois Corvos Cervejeira, the beer with the most reviews is {beer_name}, with {number_reviews} reviews.")

From the brewery Dois Corvos Cervejeira, the beer with the most reviews is Finisterra, with 10 reviews.


#### 6. On average how many different beer styles does each brewery produce?

In [44]:
# Selecting the path between the "BREWERIES" node and the "STYLE" node, including all the nodes
# and relationship types in between. With a "WITH" statement we group by each "BREWERIES" node
# the number of distinct beer styles related with that specific brewery. Then, finally we
# return the average of the distinct beer styles per brewery.

query = """
        MATCH (b1:BREWERIES)-[]->(b2:BEERS)-[h:HAS_STYLE]->(s:STYLE)
        WITH b1, COUNT(DISTINCT(s)) as diff_styles_brewery
        RETURN AVG(diff_styles_brewery) as avg_styles
    """

result = execute_read(driver, query)
avg_styles=result[0]["avg_styles"]

print(f"On average each brewery produces {round(avg_styles,2)} different beer styles.")

On average each brewery produces 10.6 different beer styles.


#### 7. Which brewery produces the strongest beers according to ABV?

In [47]:
# Selecting the nodes "BREWERIES" and "BEERS", including the relationship type along the way,
# then we are grouping by the brewery name and returning the average alcohol level of all the beers
# belonging to that "BREWERIES" node. Import to use the function toFloat as the alcohol levels of each
# beer stored in the property "abv" of each node "BEERS" are strings and we want to use the function 
# AVG() on them, and so we have to tranform them to floats. Then we are going to have a look at which
# beers are in the brewery with the strongst beers, to see if it makes sense.

query = """
        MATCH (b1:BREWERIES)-[]->(b2:BEERS)
        RETURN b1.name as brewery_name, AVG(toFloat(b2.abv)) as avg_abv
        ORDER BY COALESCE(avg_abv,0) DESC
        LIMIT 1
    """

result = execute_read(driver, query)
brewery_name=result[0]["brewery_name"]
avg_abv=result[0]["avg_abv"]

print(f"The brewery {brewery_name} is the one that produces the strongest beers, with an average of {round(avg_abv,2)}% ABV.")

The brewery 1648 Brewing Company Ltd is the one that produces the strongest beers, with an average of 25.58% ABV.


In [66]:
# Let's have a look which are the beers that this brewery "1648 Brewing Company Ltd" produces
# and their respective abv. As we can see 2 of the beers that "1648 Brewing Company Lts" produces
# according to the data have 100% alcohol, which is not possible for a beer. Because of that
# we are going to have a look at the brewery with second strongest beers to see if the same happens.

query = """
        MATCH (b1:BREWERIES{name:"1648 Brewing Company Ltd"})-[]->(b2:BEERS)
        RETURN b2.name as beer_name, b2.abv as alcohol
        ORDER BY alcohol DESC
    """

result = execute_read(driver, query)

for i in range(len(result)):
    beer_name=result[i]["beer_name"]
    avg_abv=result[i]["alcohol"]
    print(f"The beer {beer_name} has {avg_abv}% of alcohol.")

The beer Black Velvet has 5.5% of alcohol.
The beer Winter Warrant has 4.8% of alcohol.
The beer Signature has 4.5% of alcohol.
The beer 1648 Armistice has 4.2% of alcohol.
The beer 1648 Original has 3.9% of alcohol.
The beer Hop Pocket has 3.7% of alcohol.
The beer Ruby Mild has 3.6% of alcohol.
The beer Radiohead - OK Computer has 100.0% of alcohol.
The beer Earache: World's Shortest Album has 100.0% of alcohol.


In [52]:
# Doing the same exercise as before but now having a look at the brewery with second strongest
# beers, as the previous results are not valid.

query = """
        MATCH (b1:BREWERIES)-[]->(b2:BEERS)
        RETURN b1.name as brewery_name, AVG(toFloat(b2.abv)) as avg_abv
        ORDER BY COALESCE(avg_abv,0) DESC
        LIMIT 2
    """

result = execute_read(driver, query)
brewery_name=result[1]["brewery_name"]
avg_abv=result[1]["avg_abv"]

print(f"The brewery {brewery_name} is the one that produces the strongest beers, with an average of {round(avg_abv,2)}% ABV.")

The brewery Schorschbräu is the one that produces the strongest beers, with an average of 22.52% ABV.


In [67]:
# From the output we can see that although some of the beers have high levels of alcohol
# they all make sense. So our final answer is that the brewery that produces beers with a higher level
# of alcohol is "Schorschbräu".

query = """
        MATCH (b1:BREWERIES{name:"Schorschbräu"})-[]->(b2:BEERS)
        RETURN b2.name as beer_name, b2.abv as alcohol
        ORDER BY alcohol DESC
    """

result = execute_read(driver, query)

for i in range(len(result)):
    beer_name=result[i]["beer_name"]
    avg_abv=result[i]["alcohol"]
    print(f"The beer {beer_name} has {avg_abv}% of alcohol.")

The beer Schorschbräu Schorschbock 57% has 57.5% of alcohol.
The beer Schorschbräu Schorschbock 43% has 43.0% of alcohol.
The beer Schorschbräu Dunkles has 4.9% of alcohol.
The beer Schorschbräu Schorschbock 40% has 39.44% of alcohol.
The beer Schorschbräu Schorschbock 31% has 30.86% of alcohol.
The beer Schorschbräu Schorschbock 20% has 20.0% of alcohol.
The beer Schorschbock has 16.0% of alcohol.
The beer Schorsch Weizen 16% has 16.0% of alcohol.
The beer Schorschbock Ice 13 has 13.0% of alcohol.
The beer Schorschbräu Donner Weizen has 13.0% of alcohol.
The beer Schorschweizen 13% has 13.0% of alcohol.
The beer Schorschbock 13% has 13.0% of alcohol.
The beer Schorschbräu Donner Bock has 13.0% of alcohol.


#### 8. If I typically enjoy a beer due to its aroma and appearance, which beer style should I try?

In [8]:
# With the node "BEERS" in the middle, we are creating a connection with the nodes "STYLE" and "REVIEWS".
# Then we are grouping by each syle name which is the property "name" of the "STYLE" node, and with it we
# are returning the number of different beers produced with that style, the total number of reviews
# related to these beers, the average smell rating of all these beers, the average look rating of all these
# beers, and finally a combined rating of both, which will be our metric to determine which beer to recommend.

query = """
        MATCH (s:STYLE)<-[]-(b:BEERS)-[]->(r:REVIEWS)
        RETURN s.name as style_name , count(distinct(b)) as number_beers, count(distinct(r)) as number_reviews, AVG(toFloat(r.look)) as average_look,
        AVG(toFloat(r.smell)) as average_smell, AVG(toFloat(r.look))+AVG(toFloat(r.smell)) as combined_rating
        ORDER BY COALESCE(combined_rating,0) DESC, number_beers DESC
        LIMIT 5
    """

result = execute_read(driver, query)
style_name=result[0]["style_name"]
number_beers=result[0]["number_beers"]
number_reviews=result[0]["number_reviews"]
average_smell=round(result[0]["average_smell"],2)
average_look=round(result[0]["average_look"],2)
combined_rating=round(result[0]["combined_rating"],2)
print(f"The beer style you should try is {style_name}, as there are {number_beers} different beers with this style, reviewed by {number_reviews} people, giving an average look of {average_look} and an average smell of {average_smell}, resulting on a combined rating of {combined_rating}.")

The beer style you should try is New England IPA, as there are 1919 different beers with this style, reviewed by 133642 people, giving an average look of 4.38 and an average smell of 4.41, resulting on a combined rating of 8.8.


#### 9. Using Graph Algorithms answer **one** of the following questions:
    1. Which two countries are most similiar when it comes to their **top 10** most produced Beer styles?
    2. Which beer is the most influential when considering beers are conected by users who review them?
    3. Users are connected together by their reviews to beers, taking into consideration the "overall" score they review as a weight, how many communities are formed from these relationships? How many users has the biggest community?

In [5]:
# We chose the Option 2. - Which beer is the most influential when considering
# beers are conected by users who review them? The purpose of this cell of code
# is to delete any previously created sub-graph with the name "exercise_9.2".

try:
    query = """
            CALL gds.graph.drop('exercise_9.2') YIELD graphName;
        """

    result = execute_read(driver, query)

    pprint(result)
except Exception as e:
    pprint(e)

ClientError('Failed to invoke procedure `gds.graph.drop`: Caused by: java.util.NoSuchElementException: Graph with name `exercise_9.2` does not exist on database `neo4j`. It might exist on another database.')


In [6]:
# To apply the PageRank centrality algorithm, we first create a sub-graph focused on beers.
# Our main assumption is that the more beers that are linked to a particular beer, the more
# important that beer is. To create these links between beers, we use the "REVIEWS" and "USER" nodes
# in the middle. This allows us to establish connections between beers based on the reviews and users
# associated with them.

# By using this approach, we can determine the importance of each beer in the sub-graph. Specifically,
# the more frequently a random beer leads to a specific beer, the more important that specific beer is
# deemed to be in our analysis. Through this process, we can generate a ranked list of beers based on their
# importance within the sub-graph.

# Given the large size of the database, we have decided to limit the sub-graph query to 1 billion lines.
# We believe that this subset of data will be sufficiently representative of the overall dataset and
# will allow us to obtain meaningful results from our analysis.


try:
    query = """
        CALL gds.graph.project.cypher(
            'exercise_9.2',
            'MATCH (b:BEERS) RETURN id(b) AS id',
            'MATCH (b1:BEERS)-[]->(r1:REVIEWS)-[]->(u1:USER)<-[]-(r2:REVIEWS)<-[]-(b2:BEERS)
            RETURN id(b1) AS source, id(b2) AS target
            LIMIT 1000000000'
            )
        """

    result = execute_read(driver, query)

    pprint(result)
except Exception as e:
    pprint(e)

[<Record nodeQuery='MATCH (b:BEERS) RETURN id(b) AS id' relationshipQuery='MATCH (b1:BEERS)-[]->(r1:REVIEWS)-[]->(u1:USER)<-[]-(r2:REVIEWS)<-[]-(b2:BEERS)\n            RETURN id(b1) AS source, id(b2) AS target\n            LIMIT 1000000000' graphName='exercise_9.2' nodeCount=358873 relationshipCount=1000000000 projectMillis=1723500>]


In [7]:
# After creating the sub-graph, we apply the PageRank algorithm to determine the influence
# score of each "BEERS" node in the graph. We retrieve the ID of each "BEERS" node and its
# corresponding influence score. Using reverse engineering, we obtain the property name
# associated with each node ID and also retrieve its influence score.

# Based on these results, we can identify the most influential beer in the sub-graph.


query = """
        CALL gds.pageRank.stream('exercise_9.2')
            YIELD nodeId, score
            RETURN gds.util.asNode(nodeId).name AS name, score
            ORDER BY score desc
            LIMIT 1
        """

result = execute_read(driver, query)
beer = result[0][0]
score = result[0][1]

print(f"When considering users who review the beers, the beer with the biggest influence is {beer}, with a score of {round(score,2)}.")



When considering users who review the beers, the beer with the biggest influence is Breakfast Stout, with a score of 14.41.


#### 10. Using Graph Algorithms answer **one** of the following questions:
    1. Which beer has the most similar reviews as the beer `Super Bock Stout`?
    2. Which user is the most influential when it comes to reviews made?

In [8]:
# We chose the Option 2. Which user is the most influential when it comes to
# reviews made? The purpose of this cell of code is to delete any previously
# created sub-graph with the name "exercise_10.2".

try:
    query = """
            CALL gds.graph.drop('exercise_10.2') YIELD graphName;
        """

    result = execute_read(driver, query)

    pprint(result)
except Exception as e:
    pprint(e)

ClientError('Failed to invoke procedure `gds.graph.drop`: Caused by: java.util.NoSuchElementException: Graph with name `exercise_10.2` does not exist on database `neo4j`. It might exist on another database.')


In [9]:
# To apply the PageRank centrality algorithm, we first create a sub-graph focused on the users.
# Our main assumption is that the more users that are linked to a specific user, the more important
# that user is. To create these links between users, we use the "REVIEWS" and "BEERS" nodes in the middle.
# This allows us to establish connections between beers based on the reviews and beers associated with
# them.

# By using this approach, we can determine the importance of each user in the sub-graph. Specifically,
# the more frequently a random user leads to a specific user, the more important that specific user is
# deemed to be in our analysis. Through this process, we can fenerate a ranked list of users based on
# their importance within the sub-graph.

# Given the large size of the database, we have decided to limit the sub-graph query to 1 billion lines.
# We believe that this subset of data will be sufficiently representative of the overall dataset and
# will allow us to obtain meaningful results from our analysis.

try:
    query = """
        CALL gds.graph.project.cypher(
            'exercise_10.2',
            'match (n:USER) return id(n) AS id',
            'MATCH (u1:USER)<-[]-(r1:REVIEWS)<-[]-(b1:BEERS)-[]->(r2:REVIEWS)-[]->(u2:USER)
             RETURN id(u1) AS source, id(u2) AS target
             LIMIT 1000000000'
            )
        """

    result = execute_read(driver, query)

    pprint(result)
except Exception as e:
    pprint(e)

[<Record nodeQuery='match (n:USER) return id(n) AS id' relationshipQuery='MATCH (u1:USER)<-[]-(r1:REVIEWS)<-[]-(b1:BEERS)-[]->(r2:REVIEWS)-[]->(u2:USER)\n             RETURN id(u1) AS source, id(u2) AS target\n             LIMIT 1000000000' graphName='exercise_10.2' nodeCount=164935 relationshipCount=1000000000 projectMillis=2455889>]


In [10]:
# After creating the sub-graph, we apply the PageRank algorithm to determine the influence score
# of each "USER" node in the graph. We retrieve the ID of each "USER" node and its corresponding
# influence score. Using reverse engineering, we obtain the property name associated with each node
# ID and also retrieve its influence score. 

# Based onn these results, we can identify the most influential user in the sub-graph.


query = """
        CALL gds.pageRank.stream('exercise_10.2')
            YIELD nodeId, score
            RETURN gds.util.asNode(nodeId).name AS name, score
            ORDER BY score desc
            LIMIT 1
        """

result = execute_read(driver, query)
user_name = result[0][0]
score = result[0][1]

print(f"Taking into consideration the reviews made, the user {user_name} is the one with the biggest influence, obtaining a score of {round(score,2)}.")

Taking into consideration the reviews made, the user kjkinsey is the one with the biggest influence, obtaining a score of 836.79.


#### 11. If you had to pick 3 beers to recommend using only this database, which would you pick and why?

In [10]:
# To recommend three beers, we have certain criteria that must be met.
# Firstly, they must have the highest average overall score achievable, and secondly,
# they must have received at least 100 reviews. Furthermore, the beers must be from different
# countries and breweries. Finally, it is important to note that the recommended beers must
# not be retired, as we cannot recommend a beer for tasting if it is no longer available.

query = """
        MATCH (c1:COUNTRIES)<-[]-(c2:CITIES)<-[]-(b1:BREWERIES)-[]->(b2:BEERS)-[]->(r:REVIEWS)
        WITH b2.name as beer_name, COUNT(r) as number_reviews, AVG(toFloat(r.overall)) as avg_overall,
        c1.name as country, c2.name as city, b2.retired as retired
        WHERE number_reviews > 100 and retired = "f"
        RETURN beer_name, number_reviews, avg_overall, country, city
        ORDER BY avg_overall DESC
        LIMIT 100
    """

result = execute_read(driver, query)

countries=[]
beers=[]
reviews=[]
scores=[]

for i in range(len(result)):
    if len(countries)!=3:
        if result[i]["country"] not in countries:
            countries.append(result[i]["country"])
            beers.append(result[i]["beer_name"])
            reviews.append(result[i]["number_reviews"])
            scores.append(result[i]["avg_overall"])
            
print("The beers we would recommend based on our criteria are the following:")
print(f"The beer {beers[0]} from the {countries[0]}, which has an overall score of {round(scores[0],2)} based on {reviews[0]} reviews.")
print(f"The beer {beers[1]} from {countries[1]}, which has an overall score of {round(scores[1],2)} based on {reviews[1]} reviews.")
print(f"The beer {beers[2]} from {countries[2]}, which has an overall score of {round(scores[2],2)} based on {reviews[2]} reviews.")

The beers we would recommend based on our criteria are the following:
The beer Kentucky Brunch Brand Stout from the US, which has an overall score of 4.85 based on 702 reviews.
The beer Drie Fonteinen Zenne Y Frontera from BE, which has an overall score of 4.74 based on 250 reviews.
The beer Heady Topper from GB, which has an overall score of 4.68 based on 14282 reviews.
